In [1]:
import pandas as pd
from glob import glob
import os
from PIL import Image
import numpy as np
import random

In [2]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

random.seed(34)

In [9]:
# Скачайте полные данные отсюда https://www.kaggle.com/c/dogs-vs-cats/data (примерно 820мб)
# У вас должно быть две папки train и test. Однако в данном задании мы не будем использвать test выборку
# Функцию для чтения данных мы реализовали за вас

image_paths_list = glob(os.path.join('data', 'train', '*.jpg'))

def read_1000(batch):
    X = []
    y = []
    pics = []

    for i in range(1000):
        image_name = os.path.basename(image_paths_list[i + 1000 * batch])
        image_name_parts = image_name.split('.')

        label = image_name_parts[0] if len(image_name_parts) == 3 else None
        if label:
            y.append(int(label == 'cat'))

        x = image.img_to_array(image.load_img(image_paths_list[i + 1000 * batch], target_size=(224, 224)))
        pics.append(x.flatten())

    pics = preprocess_input(np.array(pics))

    return np.array(X), np.array(y)

def read_dataset():
    X = np.array([])
    y = np.array([])

    for i in range(25):
        feats, targets = read_1000(i)
        X = np.concatenate([X, feats], axis=0)
        y = np.concatenate([y, targets], axis=0)

    return X, y

Перепишите код read_dataset() таким образом чтобы читать изображения пачками по 1000 и сразу применять предобученный ResNet на них, сохраняя только полученные вектора как множество X. Примените эту функцию на всём датасете 25000 изображений.

In [10]:
# Используйте функцию read_dataset чтобы получить обучающую выборку
features, target = read_dataset()

# Проверьте размерности загруженных данных
print(len(features), len(target))
len(features) == len(target)


0 25000


False

In [40]:
# с помощью функции train_test_split поделите выборку на train и test в отношении 70/30
# В качестве признаков используйте эмбеддинги, полученные на предыдущем шаге

features_train, features_test, target_train, target_test = train_test_split(features, target, train_size=0.7, test_size=0.3, random_state=42)
features_train

array([[[[-53.939003, -42.779   , -37.68    ],
         [-51.939003, -40.779   , -35.68    ],
         [-47.939003, -36.779   , -31.68    ],
         ...,
         [136.061   , 123.221   , 116.32    ],
         [142.061   , 129.22101 , 122.32    ],
         [146.061   , 133.22101 , 126.32    ]],

        [[-53.939003, -42.779   , -37.68    ],
         [-51.939003, -40.779   , -35.68    ],
         [-47.939003, -36.779   , -31.68    ],
         ...,
         [136.061   , 123.221   , 116.32    ],
         [142.061   , 129.22101 , 122.32    ],
         [146.061   , 133.22101 , 126.32    ]],

        [[-52.939003, -41.779   , -36.68    ],
         [-50.939003, -39.779   , -34.68    ],
         [-46.939003, -35.779   , -30.68    ],
         ...,
         [136.061   , 123.221   , 116.32    ],
         [140.061   , 127.221   , 120.32    ],
         [143.061   , 130.22101 , 123.32    ]],

        ...,

        [[ 78.061   ,  74.221   ,  81.32    ],
         [ 78.061   ,  74.221   ,  81.32    ]

In [41]:
# Создаём модель для получения эмбеддингов
model = ResNet50(weights='imagenet')

# Используя объект model постройте эмбеддинги для обучающей выборки
preds = model.predict(features_train)

Создайте объект XGBClassifier со стандартными параметрами

https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

Обучите его

In [42]:
# Получите предсказания на тестовом и на обучающем множестве
clf = XGBClassifier()
clf.fit(features_train, target_train)


/home/ultra/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ValueError: y should be a 1d array, got an array of shape (1, 1000) instead.

In [ ]:
# С помощью функции accuracy_score оцените результаты
accuracy_score(target_test, clf.predict(features_test))

Почитайте про параметры XGBoost здесь: https://sites.google.com/view/lauraepp/parameters
Сделайте тюнинг гиперпараметров для XGBoost:
1. Подберите максимальный num_iterations при котором скор на валидации близок к максимальному, а скорость обучения ещё приемлимая
2. Для заданного num_iterations подберите подходящий learning rate
3. С помощью библиотеки hyperopt подберите остальные гиперпараметры при фиксимрованных num_iterations и learning_rate. Не подбирайте параметр early_stopping

Оцените важность признаков с помощью поля feature_importances_

Пользуясь списком https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a посмотрите, какие классы оказались наиболее важными (со значениями importance больше 0)
    
Изменяйте параметры, чтобы оставить только действительно важные признаки